## Stream de Dados do Twitter com MongoDB, Pandas e Scikit Learn


#### Preparando a conexão com Twitter

In [ ]:
# Baixando pacote tweepy
!pip install tweepy

In [1]:
# Importando os módulos Tweepy, Datetime e JSON
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from datetime import datetime
import json

In [2]:
# Adicionando a API Key
consumer_key = 'xxxxxxxxxxxxxxxxxx'

In [3]:
# Adicionando a API Secret Key
consumer_secret = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'

In [4]:
# Adicionando Access Token
access_token = 'xxxxxxxxxxxxxxxxxxxxxxx'

In [5]:
# Adicionando Access Token Secret
access_secret = 'xxxxxxxxxxxxxxxxxxxxxxxxxx'

In [6]:
# Criando as chaves de autenticação
auth = OAuthHandler(consumer_key, consumer_secret)

In [7]:
auth.set_access_token(access_token, access_secret)

In [8]:
# Criando uma classe para capturar os stream de dados do Twitter e 
# armazenar no MongoDB
class MyListener(StreamListener):
    def on_data(self, dados):
        tweet = json.loads(dados)
        created_at = tweet["created_at"]
        id_str = tweet["id_str"]
        text = tweet["text"]
        obj = {"created_at": created_at, "id_str":id_str, "text":text}
        tweetind = col.insert_one(obj).inserted_id
        print(obj)
        return True

In [9]:
# Criando o objeto mylistener
mylistener = MyListener()

In [10]:
# Criando o objeto mystream
mystream = Stream(auth, listener = mylistener)

#### Preparando a Conexão com o MongoDB

In [11]:
# Importando do PyMongo o módulo MongoClient
from pymongo import MongoClient

In [12]:
# Criando a conexão ao MongoDB
client = MongoClient('localhost', 27017)

In [13]:
# Criando o banco de dados twitterdb
db = client.twitterdb

In [14]:
# Criando a collection 'col'
col = db.tweets

In [15]:
# Criando uma lista de palavras chave para buscar nos Tweets
keywords = ['Big Data', 'Python', 'Data Mining', 'Data Science']

#### Coletando os Tweets

In [16]:
# Iniciando o filtro e gravando os tweets no MongoDB
mystream.filter(track=keywords)

{'created_at': 'Mon Apr 19 12:04:42 +0000 2021', 'id_str': '1384115739267780612', 'text': 'RT @sayhitojustin1: Good morning. DO YOU WANT A CHART FOR YOUR CITIES #PPP RECIPIENTS? Comment your city below and I will make it happen. H…', '_id': ObjectId('607d71dce510e171d8d67e8e')}
{'created_at': 'Mon Apr 19 12:04:42 +0000 2021', 'id_str': '1384115739687223296', 'text': 'RT @LiveSmart: Are you ready for our BIG reveal? Don’t miss out on this unique, immersive #SmartBTS experience. 👌🏻\n\nVisit https://t.co/hRSJ…', '_id': ObjectId('607d71dde510e171d8d67e8f')}
{'created_at': 'Mon Apr 19 12:04:44 +0000 2021', 'id_str': '1384115746272202755', 'text': 'https://t.co/6scQkjTqe7', '_id': ObjectId('607d71dee510e171d8d67e90')}
{'created_at': 'Mon Apr 19 12:04:44 +0000 2021', 'id_str': '1384115747362799621', 'text': 'RT @Connected_Data: Hello world\n\nWhat an event it was! #CDLGraphAIMeetup\xa0was our biggest yet - until the next one\n\nMissed it? Want to see i…', '_id': ObjectId('607d71dee510e171d8d6

ProtocolError: ('Connection broken: IncompleteRead(0 bytes read)', IncompleteRead(0 bytes read))

In [17]:
mystream.disconnect()

In [18]:
# Verificando um documento no collection
col.find_one()

{'_id': ObjectId('607b48af4b076a297a6c79fc'),
 'created_at': 'Sat Apr 17 20:44:25 +0000 2021',
 'id_str': '1383521756091011086',
 'text': 'RT @BenjaminP3ters: Datos Health Partners With Medial EarlySign, Augments Artificial Intelligence With Remote Care Automation To Optimize P…'}

#### Análise de Dados com Pandas e Scikit-Learn

In [19]:
# Criando um dataset com dados retronados do MongoDB
dataset = [{"created_at": item["created_at"], "text": item["text"],} for item in col.find()]

In [20]:
# Importando pandas
import pandas as pd

In [21]:
# Criando dataframe 
df = pd.DataFrame(dataset)

In [22]:
df

,created_at,text
0,Sat Apr 17 20:44:25 +0000 2021,RT @BenjaminP3ters: Datos Health Partners With...
1,Sat Apr 17 20:44:27 +0000 2021,Get quality grades for your online classes.\n\...
2,Sat Apr 17 20:44:28 +0000 2021,RT @IainLJBrown: Artificial Intelligence has c...
3,Sat Apr 17 20:44:29 +0000 2021,Machine Learning Pocket Reference: Working wit...
4,Sat Apr 17 20:44:29 +0000 2021,RT @alchemist9007: #PROGRAMMER #HTML #PHP #GRA...
...,...,...
357,Mon Apr 19 12:04:48 +0000 2021,RT @RobinsonAmandaL: OSU post doc position for...
358,Mon Apr 19 12:04:48 +0000 2021,RT @ben_nuttall: It's been a year since we los...
359,Mon Apr 19 12:04:51 +0000 2021,RT @AssignmentTeam3: Grade A+ assured in any p...
360,Mon Apr 19 12:04:55 +0000 2021,RT @mytrashcode: No. 640 programming meme #pro...


In [35]:
# Importando scikit learn
from sklearn.feature_extraction.text import CountVectorizer

In [36]:
# Usando CountVectorizer para criar uma matriz de documentos
cv = CountVectorizer()
count_matrix = cv.fit_transform(df.text)

In [37]:
# Contando o número de ocorrências das principais palavras em nosso dataset
word_count = pd.DataFrame(cv.get_feature_names(), columns=["word"])
word_count["count"] = count_matrix.sum(axis=0).tolist()[0]
word_count = word_count.sort_values("count", ascending=False).reset_index(drop=True)
word_count[:50]

,word,count
0,rt,250
1,https,210
2,co,200
3,to,100
4,python,98
5,the,85
6,data,71
7,and,70
8,of,66
9,read,62
